# Interface resisitivity

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [ ]:
inner = Rectangle(1,1).Face()
inner.edges.name="interface"
inner.edges.Max(X).name="interface_right"

outer = MoveTo(-1,-1).Rectangle(3,3).Face()
outer.edges.name="dir"
outer = outer-inner

inner.faces.name="inner"
outer.faces.name="outer"
geo = Glue ([inner,outer])
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.2))

Draw (mesh)

In [ ]:
print (mesh.GetMaterials(), mesh.GetBoundaries())

# Using independent $H^1$ spaces on sub-domains

In [ ]:
fesi = H1(mesh, order=3, definedon=mesh.Materials("inner"))
feso = H1(mesh, order=3, definedon=mesh.Materials("outer"), dirichlet="dir")
fes = fesi*feso

(ui,uo), (vi,vo) = fes.TnT()

In [ ]:
a = BilinearForm(fes)
a += grad(ui)*grad(vi)*dx("inner")
a += grad(uo)*grad(vo)*dx("outer")
a += 10 * (ui-uo) * (vi-vo) * ds("interface")
a.Assemble()

f = LinearForm(1*vi*dx("inner")).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec

sol = mesh.MaterialCF({"inner":gfu.components[0], "outer":gfu.components[1]})
Draw (sol, mesh);

In [ ]:
fes1 = H1(mesh, order=3, dirichlet="dir")

fesext = H1(mesh, order=3, definedon="outer")
actdofs = fesext.GetDofs(mesh.Boundaries("interface_right")) & \
    ~fesext.GetDofs(mesh.Boundaries("interface"))
fesext = Compress (fesext, active_dofs=actdofs)
print (actdofs)

In [ ]:
fes = fes1 * fesext
(u,uext), (v,vext) = fes.TnT()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx("inner")
a += (grad(u)+grad(uext))* (grad(v)+grad(vext))*dx("outer")
a += 5 * uext*vext * ds("interface_right")
a.Assemble()

f = LinearForm(1*v*dx("inner")).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec

sol = mesh.MaterialCF({"inner":gfu.components[0], "outer":gfu.components[0]+gfu.components[1]})
Draw (sol, mesh);